# Importing libraries and loading data

In [ ]:
import matplotlib.pyplot as plt
import os
import pandas as pd
from sklearn import preprocessing
from datetime import datetime as dt
from tensorflow.python import keras 
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from IPython.display import IFrame
import ipywidgets as widgets
from sklearn.preprocessing import MinMaxScaler
from utils import *
import datetime

# Loading model

In [ ]:
update_one_hot_data('.././data/combined_data_22-jun.tsv','.././data/combined_data_22-jun_one_hot_data.tsv')
df_new_data = pd.read_csv('.././data/combined_data_22-jun_one_hot_data.tsv',
                 sep='\t',
                 header=0)
start_size=df_new_data.shape[0]
print(df_new_data.shape)
df_new_data = remove_outlier(df_new_data, "bike_count")
print("We have removed "+str(start_size-df_new_data.shape[0])+" outliers tuples (which is "+str((start_size-df_new_data.shape[0])/start_size*100)+" % of total).")
df_new_data=remove_features(df_new_data)    
df_new_data.set_index('timestamp_from', inplace=True)
df_new_data = df_new_data.sort_values(by=['timestamp_from'])
print(df_new_data.shape)


#set bike_count as Y
df_y = df_new_data.bike_count
df_x = df_new_data.drop(columns="bike_count")


#Normalization
x = df_x.values
y = df_y.values.reshape(-1, 1)
print(y.shape)
x_scaler = preprocessing.MinMaxScaler()
x_normalized = x_scaler.fit_transform(x)

y_scaler = preprocessing.MinMaxScaler()
y_normalized = y_scaler.fit_transform(y)

x_in = pd.DataFrame(x_normalized)
y_in = pd.DataFrame(y_normalized)




new_model = keras.models.load_model('.././model/model-23-jun.bin')
new_model.summary()
evaluate_model_and_show_graph(x_in,y_in,new_model,y_scaler)


# Show bike poles map

In [ ]:
if not os.path.isfile('.././map/bike_Brussels-points-map.html'):
    from map.generate_map import GENERATE_MAP_MAIN_FUNCTION
    GENERATE_MAP_MAIN_FUNCTION()
IFrame(src='.././map/bike_Brussels-points-map.html',width=900,height=600)


# Select calendar

In [ ]:
print("Select a start and end date")
start_date=datetime.datetime.now()
end_date=datetime.datetime.now()
@widgets.interact(start=widgets.DatePicker(),value='2019-06-19')
def foo(start=datetime.datetime.now()):
    global start_date
    start_date=start
    return str(start)
@widgets.interact(end=widgets.DatePicker())
def foo2(end=datetime.datetime.now()):
    global end_date
    end_date=end
    return str(end)

# Gather information

In [ ]:

if start_date ==None or end_date==None:
    print("You need to set a start and end date")
else:
    if start_date==end_date:
        print("The dates must be different.")
    elif end_date<start_date or datetime.datetime.now()>datetime.datetime(start_date.year,start_date.month,start_date.day):
        print("Start date needs to be lower than end date and start date needs to be greater than today.")
    else:
        obtain_next_dates_data(start_date,end_date)


# Predict

In [ ]:
update_one_hot_data('.././data/next_dates_data.tsv','.././data/next_dates_data_one_hot_data.tsv')
df_next_days_data = pd.read_csv('.././data/next_dates_data_one_hot_data.tsv',
                 sep='\t',
                 header=0)
start_size=df_next_days_data.shape
print(start_size)
df_next_days_data=remove_features(df_next_days_data)
df_next_days_data.set_index('timestamp_from', inplace=True)
df_next_days_data = df_next_days_data.sort_values(by=['timestamp_from'])
print(df_next_days_data.shape)

x_scaler = MinMaxScaler()

x_data=df_next_days_data.drop(columns="bike_count")
x_normalized = x_scaler.fit_transform(x_data)
x_in = pd.DataFrame(x_normalized)
bike_prediction = new_model.predict(x_in)



y_normalized = y_scaler.fit_transform(y)
predicted_values=y_scaler.inverse_transform(bike_prediction)


# Show forecasting on dynamic map

In [ ]:
import folium
import folium.plugins as plugins
import numpy as np
from pygame.math import Vector2
from datetime import datetime as dt
def points(number, center, radius):
    angle = 360/number
    
    point_list = []
    for i in range(number):
        # Create a vector with the length of the radius, rotate it
        # and add it to the center point to get a point on the circumference.
        vec = center + Vector2(radius, 0).rotate(i*angle)
        # pygame.draw.circle needs ints, so we have to convert the vector.
        point_list.append([float(vec.x), float(vec.y),1])
    return point_list
def convert_to_proper_list(lat_list,long_list):
    return_list=[]
    for i in range(0,len(lat_list)):
        return_list.append([lat_list[i],long_list[i]])
    return return_list    
def generate_circle_points(lat,long,number_points,limit_lat=0.00007,limit_long=0.0001):
    return_list=[]
    
    for i in range(0,number_points):
        return_list.extend(points(number_points,(lat,long),0.0001*i))
        
    
    return return_list
    
def convert_row_to_multiple_points(data):
    import random 
    rand_limits=0.0002
    return_list=[]
    for row in data:
        temp_row=[]
        for tuple_r in row:            
            for i in range(0,tuple_r[2]):
                #print(random.uniform(-rand_limits,rand_limits))
                temp_row.extend(generate_circle_points(tuple_r[0],tuple_r[1],i))
                #temp_row.append([tuple_r[0]+random.uniform(-rand_limits*i,rand_limits*i),tuple_r[1]+random.uniform(-rand_limits*i,rand_limits*i),i])            
        return_list.append(temp_row)
    return return_list            
def SHOW_DYNAMIC_MAP(x_data,predicted_num_bikes):
    import random    
    import statistics as st
    data_information=[]
    latitudes_list=[]
    longitudes_list=[]
    timestamp_list=[]
    rand_limits=0    
    for row in x_data.index:
        timestamp_list.append(row)
    for row in x_data.latitude:
        latitudes_list.append(row+random.uniform(-rand_limits,rand_limits))
    for row in x_data.longitude:
        longitudes_list.append(row+random.uniform(-rand_limits,rand_limits))  
    for i in range(0,len(latitudes_list)):
        data_information.append([latitudes_list[i],longitudes_list[i],float(predicted_num_bikes[i])])             
    data=[]   
    for i in range(0,int(len(timestamp_list)/9)):
        data.append(convert_to_proper_list(latitudes_list[i*9:(i*9)+9],longitudes_list[i*9:(i*9)+9]))              
    weight = 1  # default value
    i=0
    
    for time_entry in data:
        for row in time_entry:
            row.append(int(predicted_num_bikes[i]))
            i+=1
    
    from datetime import datetime, timedelta   
    time_index = [
        (dt.fromtimestamp(k)).strftime('%Y-%m-%d-%H:%M:%S') for k in timestamp_list
    ]       
    time_index=list(np.unique(np.array(time_index)))    
    
    #print(predicted_num_bikes)
    #input()
    #print(time_index)
    #print(data[23])
    #print(data[24])
    data=convert_row_to_multiple_points(data)   
    
    m = folium.Map([st.mean(latitudes_list), st.mean(longitudes_list)], tiles='OpenStreetMap', zoom_start=11.5)
    plugins.Fullscreen(
    position='topright',
    title='Expand me',
    title_cancel='Exit me',
    force_separate_button=True
    ).add_to(m)
    hm = plugins.HeatMapWithTime(
        data,
        index=time_index,
        auto_play=True,
        max_opacity=0.4
    )

    hm.add_to(m)

    m.save('.././map/dynamic_forecasting_ap.html')
SHOW_DYNAMIC_MAP(x_data,predicted_values)
IFrame(src='.././map/dynamic_forecasting_ap.html',width=900,height=600)